# Utilisation des expressions régulières dans la récupération de données

Les expressions régulières permettent de chercher des motifs dans le texte et cette manière de faire peut être utile dans la récupération des informations afin de former une base de données.


## Aujourd'hui : récupérer les informations sur une page Web

Récupérer les infromations, sur Internet par exemple, revient à ces trois étapes :

![title](scraping.jpeg)

1. On part d'une page Web, souvent au format HTML, dans laquelle se trouve les informations que l'on veut récupérer
2. En fonction de cette structure, on fait ce qu'on appel du "scraping", c'est-à-dire de la récupération de données
3. Finalement, il s'agit de structurer ces données dans un format, de type tableur par exemple

## Reprenons plus précisément ces trois étapes

### 1. Structure d'une page Web

Derrière une page Web il y a du texte, du code: on passe d'une représentation, disons, visuelle et interactive à une construction textuelle. La strucutre d'une page Web ressemble le plus souvent à du HTML ou ses dérivées. Le principe de ce langage est que le texte, les informations sont contenues dans des balises.

* Les balises se trouve au début (balise ouvrante) et à la fin (balise fermante) pour enclore des informations. 
* Entre deux balises, il peut y avoir d'autres balises
* Une balise, dans sa description, peut avoir plus ou moins d'informations

![title](web.png)
![title](html.png)

Les informations qui nous intéressent se trouvent entre les balises. Il nous faut donc comprendre la structure de notre page pour pouvoir y appliquer des expressions régulières. 

### Exemple 

De quelle manière dois-je concevoir mon expression régulière si je veux récupérer le texte "Hello World" ?

![title](html2.png)

## 2. Récupération des données

Cette étape permet de lier notre page Web non-structurée à un des informations structurée de type tableur. Après avoir compris la structure de la page Web, notre script Python devra suivre ces différentes étapes:

* A partir d'une URL, télécharger les informations d'une page Web (sous la forme de l'exemple ci-dessus)
* Construire et appliquer nos expressions régulières selon les informations que l'on désire récupérer
* Créer une boucle pour récupérer toutes les infromations
* Enregistrer les données

### Exemple

Avant de passer au code, voyons à quoi cela peut ressembler

![title](python.jpeg)

## 3. Enregistrer les données

L'idée de ce processus est de passer d'une page Web non structurée à un format structuré pour la maipulaiton de données. Dans cette optique, le format dans lequel les informations seront enregistrées doit être:

* Structuré, avec des lignes et des colones clairement définies
* Interopérable, c'est-à-dire compatible et réutilisable, par exemple sous forme d'un tableur


### Exemple

Voilà à quoi ressemble un fichier au format .csv : 

* Colones séparées par des virgules (d'où le "c" de csv ; par exemple tsv se fait avec des tabulations)
* Lignes séparées par un retour à la ligne

![title](csv.png)

## Exercice : récupérer les informations des films sur IMDB

## Étape 1 - la strcuture de la page Web

L'exercice que je vous propose part d'une recherche des titres de films et séries contenant le terme "Vénus" : https://www.imdb.com/search/title/?title=venus

![title](imdb.png)
![title](imdb_html.png)

### Quelles informations pouvez-vous et voulez-vous récupérer ?

* Reste-t-on sur la première page de la requête ?
* Quelles informations ?
    * Le titre
    * La durée
    * Le genre
    * L'affiche (image)
    * ...

## Étape 2 - Créer un script pour le "scraping"

Après avoir compris la strucutre de la page Web, il faut créer un script pour les récupérer. Les étapes successivent sont:

* 2.1 Importer les libraires nécessaires
* 2.2 Requêter d'après l'URL pour récupérer les informations 
* 2.3 Construire les expressions régulières et les appliquer
* 2.4 Automatiser le processus sur toutes les pages



#### 2.1 Importer les librairies nécessaires

In [1]:
# Importer les librairies

#!conda install --yes -c anaconda requests
import requests # pour télécharger le contenu d'une URL

#!conda install --yes -c anaconda bs4 
from bs4 import BeautifulSoup as bs # pour récupérer le contenu téléchargé de l'URL

import os # pour naviguer dans les dossiers et fichiers

import re # pour les expressions régulières

#!conda install --yes -c anaconda pandas
import pandas as pd # pour manipuler des tableurs


#!conda install --yes -c conda-forge tqdm
from tqdm.notebook import tqdm # pour afficher une bar de progression

#### 2.2 Requêter d'après l'URL pour récupérer les informations 

Regardons plus précisément la structures des URLs pour pouvoir récupérer toutes les pages de notre requête sur IMDB

In [2]:
# URL de la première page

url = "https://www.imdb.com/search/title/?title=venus"

In [3]:
# Logique des URLs suivants:
# On voit que seuelement une partie de l'URL change, 
# c'est la partie qui définit le nombre de titres dans la page.
# Ainsi il suffit de changer ce nombre pour avoir les titres de la page suivante

url2 = "https://www.imdb.com/search/title/?title=venus&start=51&ref_=adv_nxt"
url3 = "https://www.imdb.com/search/title/?title=venus&start=101&ref_=adv_nxt"

Finalement, plusieurs informations nous permette de construire une liste avec tous les URLs

![title](imdbnb.png)

* Le 1er URL est différent des autres
* Les URLs suivant ont un incrément de 50 entre chaque page
* Notre requête a un maximum de 886 titres

**Avant de faire une requête, sachant toutes ces informations il faut préparer tous les URLs**

In [4]:
# Combien de pages avons-nous ?

total_titres = 886 # nombre de pages total
count = total_titres//50 # nombre de titres total divisé par le le nombre de titre par page

print("Nombre de pages :",count)

Nombre de pages : 17


In [5]:
# Préaprer une liste avec l'incrément
# Cette liste nous permettra de remplacer le nombre dans l'URL
# On commence du titre 51 jusqu'au titre 851 qui définit la dernière page

increment = [(i*50)+1 for i in range(1, (count+1))]

print(increment)

[51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601, 651, 701, 751, 801, 851]


**Maintenant, il faut créer tous les URLS**

* Prendre une URL de base
* Par une boucle, remplacer le nombre des titres par page
* Enregistrer le tout 

In [6]:
# Créer une liste avec tous les urls

# A l'aide du %d il est possible d'insérer notre incrément
url_pattern = "https://www.imdb.com/search/title/?title=venus&start=%d&ref_=adv_nxt"

# Insérer notre incrément dans notre pattern
urls = [url_pattern % v for v in increment]

print("Nombre d'URLs :", len(urls))

Nombre d'URLs : 17


In [7]:
# Ajouter l'URL de la première page
# Car le premier URL est différent !

urls.append(url)

**Maintenant que nos URLs sont prêts, il est possible de récupérer les informations dans chaque page**

Avant d'appliquer le processus sur toutes les pages, commençons par la première page. Puis il s'agira d'automatiser la tâche sur toutes les pages

In [8]:
# La requête d'un contenu Web d'après une URL se fait simplement de cette manière

res = requests.get(url)
soup = bs(res.content, 'html.parser')

**Cependant, on peut préciser notre requête, par exemple en prenant les informations contenues entre deux balises. Entre quelles balises sont contenues les films?**

In [9]:
# En observant la page de recherche, on voit que les informations de chaque film sont contenues dans la balise
# <div class="lister-item-content">

res = requests.get(url)
soup = bs(res.content, 'html.parser')
results = soup.find_all("div", class_="lister-item mode-advanced")

In [10]:
# Combien de films avons-nous par page ?

print(f"Nombre de films: {len(results)}")

Nombre de films: 50


In [11]:
# Voir les informations disponibles dans le 1er résultat
# On peut également voir les différentes informations disponibles par film
# Vous pouvez jouer avec l'indexation pour voir les autres films

results[0]

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt2406252"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt2406252/"> <img alt="La Vénus à la fourrure" class="loadlate" data-tconst="tt2406252" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BMjM0ODgyOTM1Ml5BMl5BanBnXkFtZTgwMzI5ODM4MTE@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt2406252/">La Vénus à la fourrure</a>
<span class="lister-item-year text-muted unbold">(2013)</span>
</h3>
<p class="text-muted">
<span class="certificate">16</span>
<span class="ghost">|</span>
<span class="runtime">96 min</span>
<span class="ghost">|</span>
<span class="genre">
Drama            </span>
</p>
<div class

#### 2.3 Construire les expressions régulières et les appliquer

D'après la structure et les informations désirée, il va falloir construire des expressions régulières pour récupérer les infromations

**Commençons par le titre du premier film, par exemple**

* Entre quelles balises est contenu le titre du film ?
* Quelle expression régulière écrire ?
* Quelle est le résultat obtenu ?

Le titre se trouve à plusieurs endroits, quelle est la manière la plus standard ?

![title](titre.png)

**Le titre est entre ces deux balises, il faut désormais écrire l'expression régulière**

In [12]:
# On utilise la libraire "re" et le module findall pour appliquer notre expression régulière

title_reg = re.findall("((<a href=\"\/title\/).*\S.(</a>))", str(results[0]))
title_reg

[('<a href="/title/tt2406252/">La Vénus à la fourrure</a>',
  '<a href="/title/',
  '</a>'),
 ('<a href="/title/tt2406252/vote" rel="nofollow" title="Delete"><span>X</span></a>',
  '<a href="/title/',
  '</a>')]

**Le titre est là mais il est encombré de texte inutile. Il faut encore un petit travail pour le nettoyer**

In [13]:
# Prendre la bonne chaîne de caractère dans la liste
# Avec l'indexation, enlever le surplus

title_reg[0]

('<a href="/title/tt2406252/">La Vénus à la fourrure</a>',
 '<a href="/title/',
 '</a>')

In [14]:
title_reg[0][0]

'<a href="/title/tt2406252/">La Vénus à la fourrure</a>'

In [15]:
title_reg[0][0][28:-4]

'La Vénus à la fourrure'

**Dans les informations récupérées il y a aussi l'identifiant du films sur IMDB**

Cette information est très importante car elle permet d'avoir un identifiant unique pour chaque film. Cela permet de lier le film à sa source (IMDB) ou à d'autres sources (reconciliation).

**Où se situe-t-il ? Comment le récupérer ?**

In [16]:
# A l'aide de split, on peut diviser la chaîne de caractère
# Puis avec l'indexation, récupérer l'identifiant

title_reg[0]

('<a href="/title/tt2406252/">La Vénus à la fourrure</a>',
 '<a href="/title/',
 '</a>')

In [17]:
title_reg[0][0]

'<a href="/title/tt2406252/">La Vénus à la fourrure</a>'

In [18]:
title_reg[0][0].split("/")

['<a href="', 'title', 'tt2406252', '">La Vénus à la fourrure<', 'a>']

In [19]:
title_reg[0][0].split("/")[2]

'tt2406252'

**Maintenant, à vous de jouer !**

* Quelles informations ?
* Comment les récupérer ?
* Quelle expression régulière ?

In [20]:
# Par exemple, la durée

duree_reg = re.findall("((<span class=\"runtime\">).*\S.(</span>))", str(results[0]))
duree_reg

[('<span class="runtime">96 min</span>', '<span class="runtime">', '</span>')]

In [21]:
# Durée

duree_reg[0][0][22:-7]

'96 min'

In [22]:
# Par exemple, le genre

genre_reg = re.findall("((genre).*\S.(\n).*\S.)", str(results[0]))
genre_reg

[('genre">\nDrama            </span>', 'genre', '\n')]

In [23]:
# Genres

genre_reg[0][0][8:-7]

'Drama            '

In [24]:
# Par exemple, le lien de l'affiche

lien_reg = re.findall("((https).*\S.(.jpg))", str(results[0]))
lien_reg

[('https://m.media-amazon.com/images/M/MV5BMjM0ODgyOTM1Ml5BMl5BanBnXkFtZTgwMzI5ODM4MTE@._V1_UX67_CR0,0,67,98_AL_.jpg',
  'https',
  '.jpg')]

In [25]:
# Liens pour l'image

lien_reg[0][0]

'https://m.media-amazon.com/images/M/MV5BMjM0ODgyOTM1Ml5BMl5BanBnXkFtZTgwMzI5ODM4MTE@._V1_UX67_CR0,0,67,98_AL_.jpg'

**Petite astuce, en faisant mes recherches j'ai vu qu'en ajoutant le suffixe ci-dessous, les images étaitent de meilleure qualité**

In [26]:
# Liens de l'image de meilleure qualité
# en ajoutant "_FMjpg_UX4000_.jpg" après V1

l = lien_reg[0][0][10:].split("V1")[0]
lien = l + str("V1_FMjpg_UX4000_.jpg")
lien

'media-amazon.com/images/M/MV5BMjM0ODgyOTM1Ml5BMl5BanBnXkFtZTgwMzI5ODM4MTE@._V1_FMjpg_UX4000_.jpg'

#### 2.4 Automatiser le processus sur toute les page

Après avoir vu quelles informations et comment les récupérer, il est possible d'automatiser la tâche sur toutes les pages. Il nous faut reprendre le travail fait précédemment:

* La liste des URLs
* Les expressions régulières 

Il faut enregistrer les informations dans un dictionnaire afin de pouvoir ensuite les transformer en tableur

In [27]:
# Vérifions nos URLs

urls

['https://www.imdb.com/search/title/?title=venus&start=51&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=101&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=151&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=201&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=251&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=301&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=351&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=401&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=451&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=501&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=551&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=601&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title=venus&start=651&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?ti

**A l'aide d'une boucle et de l'exemple de requête précédent, récupérez tous les films**

In [28]:
req = []
for u in urls:
    ...

In [29]:
# Réponse
# Tous les films

req = [] # liste vide pour y mettre nos résultats
for u in tqdm(urls): # on itère dans notre liste d'URLs pour chercher les infos sur chaque page
        res = requests.get(u) # on requête le contenu
        soup = bs(res.content, 'html.parser') # on le charge 
        results = soup.find_all("div", class_="lister-item mode-advanced") # on cherche tous les films compris entre les balises
        for t in results: # on requête dans le resultats pour avoir chaque film, film par film
            req.append(t) # les films sont enregistrés dans notre liste

  0%|          | 0/18 [00:00<?, ?it/s]

In [30]:
print(f"Nombre de films: {len(req)}")

Nombre de films: 890


**Maintenant, nous avons tous les films. Et comme sur l'exemple précédent, il faut y appliquer les expressions régulières**

Créer une boucle qui:

* Prends la liste de films 
* Récupère les informations à l'aide des expressions régulières
* Enregistre les informations dans un dictionnaire

In [31]:
req[0]

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt4163316"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt4163316/"> <img alt="Montes de Venus" class="loadlate" data-tconst="tt4163316" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BOTIxMzc4ODktNTZkZS00M2FkLWEzNTctOTcxNTE1ZWYyNzJiXkEyXkFqcGdeQXVyMzEwMDM2NDE@._V1_UY98_CR1,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">51.</span>
<a href="/title/tt4163316/">Montes de Venus</a>
<span class="lister-item-year text-muted unbold">(2006)</span>
</h3>
<p class="text-muted">
<span class="runtime">80 min</span>
<span class="ghost">|</span>
<span class="genre">
Thriller            </span>
</p>
<div class="ratings-bar">
<div class="inline-block ra

In [32]:
# Boucle pour récupérer les informations

res = [] #enregistrer les informations
ex = [] #voir les exceptions
for r in tqdm(req): # itérer dans la liste de films faite précédement
    try:  # try-excpet permet ici d'itérer indépendemment des données manquantes
        
        # selon les expressions régulières faites précédemment
        # le titre
        title_reg = re.findall("((<a href=\"\/title\/).*\S.(</a>))", str(r))
        title = title_reg[0][0][28:-4]
        
        # l'identifiant
        ide = title_reg[0][0].split("/")[2]
        
        # la durée
        duree_reg = re.findall("((<span class=\"runtime\">).*\S.(</span>))", str(r))
        duree = duree_reg[0][0][22:-7]
        
        # le genre
        genre_reg = re.findall("((genre).*\S.(\n).*\S.)", str(r))
        genre = genre_reg[0][0][8:-7]
        
        # lien pour les images
        lien_reg = re.findall("((https).*\S.(.jpg))", str(r))
        l = lien_reg[0][0].split("V1")[0]
        lien = l + str("V1_FMjpg_UX4000_.jpg")
        
        # ensuite il faut enregister les données, ici dans un dictionnaire
        # pour chaque information (valeur), il faut y joindre une clé 
        
        data = {
                "titre": title,
                "id": ide,
                "genre": genre,
                 "temps": duree,
                 "lien": lien
            }
        
        # il y a un dictionnaire par film
        # il faut ensuite les enregistrer dans une liste
        
        res.append(data) if data not in res else res #éviter de mettre deux fois le même film
    except Exception as e:
        ex.append(r) # enregistrer les exceptions (voir les erreurs ou faire une autre requête par la suite)

  0%|          | 0/890 [00:00<?, ?it/s]

In [33]:
print("Films récupérés: ", len(res), "\nFilms disponibles:", len(req))

Films récupérés:  255 
Films disponibles: 890


**Pourquoi est-ce que sur les 890 films, seulement 254 ont été récupérés ?**

Car il y a des films redondants ou des infortmations manquantes et dans notre cas nous voulions les films avec toutes ces informations. Cette méthode est assez simple, vous pouvez l'augmenter en jouant avec les autres informations disponibles et celles qui sont manquantes. 

## Étape 3 - Enregistrer les données

Maintenant que nos données ont été récupéres, il faut les structurer et les enregistrer. Pour cela ont utilise la librairie "pandas" pour que nos données prennent la forme d'un tableur.

In [34]:
# Voir les informations sous forme de Dataframe
# Toutes les entrées avec des images

df = pd.DataFrame(res) # on peut également mettre l'id en indexe
df # voir son tableur

,titre,id,genre,temps,lien
0,Montes de Venus,tt4163316,Thriller,80 min,https://m.media-amazon.com/images/M/MV5BOTIxMz...
1,>Mars Met Venus (Part Cowo),tt11997088,"Comedy, Romance",94 min,https://m.media-amazon.com/images/M/MV5BZTU4ND...
2,Bananarama: Venus,tt6696890,Music,4 min,https://m.media-amazon.com/images/M/MV5BMGQzY2...
3,I Afroditi stin avli,tt2091317,Drama,63 min,https://m.media-amazon.com/images/M/MV5BNzljNG...
4,Venus and Mars,tt3962816,Drama,90 min,https://m.media-amazon.com/images/M/MV5BODMzNz...
...,...,...,...,...,...
250,The Naked Venus,tt0144415,Drama,75 min,https://m.media-amazon.com/images/M/MV5BNjkyOG...
251,Venere imperiale,tt0057638,"Drama, History, Romance",140 min,https://m.media-amazon.com/images/M/MV5BMjA5MD...
252,American Venus,tt0783478,Drama,81 min,https://m.media-amazon.com/images/M/MV5BMWQ0ZW...
253,Venus Rising,tt0114831,"Action, Sci-Fi",91 min,https://m.media-amazon.com/images/M/MV5BMTYyMj...


In [35]:
# Changer l'index

df = df.set_index('id')
df

,titre,genre,temps,lien
id,,,,
tt4163316,Montes de Venus,Thriller,80 min,https://m.media-amazon.com/images/M/MV5BOTIxMz...
tt11997088,>Mars Met Venus (Part Cowo),"Comedy, Romance",94 min,https://m.media-amazon.com/images/M/MV5BZTU4ND...
tt6696890,Bananarama: Venus,Music,4 min,https://m.media-amazon.com/images/M/MV5BMGQzY2...
tt2091317,I Afroditi stin avli,Drama,63 min,https://m.media-amazon.com/images/M/MV5BNzljNG...
tt3962816,Venus and Mars,Drama,90 min,https://m.media-amazon.com/images/M/MV5BODMzNz...
...,...,...,...,...
tt0144415,The Naked Venus,Drama,75 min,https://m.media-amazon.com/images/M/MV5BNjkyOG...
tt0057638,Venere imperiale,"Drama, History, Romance",140 min,https://m.media-amazon.com/images/M/MV5BMjA5MD...
tt0783478,American Venus,Drama,81 min,https://m.media-amazon.com/images/M/MV5BMWQ0ZW...


In [36]:
# Enregistrer le tableur au format .csv

df.to_csv('venus_IMDB.csv')